In [1]:
from PySide6.QtWidgets import QWidget, QMainWindow, QVBoxLayout,QGridLayout, QFileDialog, QDialog, QFormLayout,QLineEdit,QPushButton
from PySide6.QtGui import QPainter, QColor, QPen, QPolygonF, QAction
from PySide6.QtCore import Qt, QPointF, QLineF
import numpy as np

In [2]:
class Ponto:
    def __init__(self,x,y) -> None:
        self.x = x
        self.y = y
        pass

    def __str__(self) -> str:
        return (f"({self.x},{self.y})")


In [3]:
class Reta:
    def __init__(self,*pontos) -> None:
        self.pontosReta = []
        for ponto in pontos:
            self.pontosReta.append(ponto)
        pass

    def __str__(self) -> str:
        return '('+','.join(str(e) for e in self.pontosReta)+')'

In [4]:
class Poligono:

    def __init__(self,*pontos) -> None:
        self.pontosPoligono = []
        for ponto in pontos:
            self.pontosPoligono.append(ponto)
        pass

    
    def __str__(self) -> str:
        return '('+','.join(str(e) for e in self.pontosPoligono)+')'

In [5]:
class Window:
    def __init__(self,minX,minY,maxX,maxY) -> None:
        self.minX = minX
        self.minY = minY
        self.maxX = maxX
        self.maxY = maxY
        self.pontos = []
        self.pontos.append(Ponto(self.minX,self.minY))
        self.pontos.append(Ponto(self.maxX,self.minY))
        self.pontos.append(Ponto(self.maxX,self.maxY))
        self.pontos.append(Ponto(self.minX,self.maxY))

        self.bordas = []
        self.bordas.append(Ponto(self.minX,self.minY))
        self.bordas.append(Ponto(self.maxX,self.minY))
        self.bordas.append(Ponto(self.maxX,self.maxY))
        self.bordas.append(Ponto(self.minX,self.maxY))
        pass

    def __str__(self) -> str:
        
        return (f"({self.minX},{self.minY},{self.maxX},{self.maxY})")

In [6]:
class Viewport:
    def __init__(self,minX,minY,maxX,maxY) -> None:
        self.minX = minX
        self.minY = minY
        self.maxX = maxX
        self.maxY = maxY
        self.pontos = []
        self.pontos.append(Ponto(self.minX,self.minY))
        self.pontos.append(Ponto(self.maxX,self.minY))
        self.pontos.append(Ponto(self.maxX,self.maxY))
        self.pontos.append(Ponto(self.minX,self.maxY))
        pass


    def __str__(self) -> str:
        
        return (f"({self.minX},{self.minY},{self.maxX},{self.maxY})")

In [7]:
class Transformada:

    def __init__(self,win,view) -> None:
        self.win = win
        self.view = view
        pass

    def transformarPonto(self,x,y) -> Ponto:
        self.pontoX = (((x-self.win.minX)/(self.win.maxX-self.win.minX))*(self.view.maxX-self.view.minX))
        self.pontoY = ((1-(y-self.win.minY)/(self.win.maxY-self.win.minY))*(self.view.maxY-self.view.minY))
        return Ponto(self.pontoX,self.pontoY)
    
    def transformarReta(self,pontos) -> Reta:
        pontosRetaTrans = Reta()
        for x in pontos:
            pontosRetaTrans.pontosReta.append(self.transformarPonto(x.x,x.y))
        return pontosRetaTrans
            
    
    def transformarPoli(self,pontos) -> Poligono:
        pontosPolygonsTrans = []
        for x in pontos:
            pontosPolygonsTrans.append(self.transformarPonto(x.x,x.y))
        return pontosPolygonsTrans
            
    def __str__(self) -> str:
        return (f"Viewport: {self.view}\nWindow: {self.win}")

In [8]:
class LerXML:
    import xml.etree.ElementTree as Et
    
    def __init__(self,file) -> None:

        tree = self.Et.parse(file) #Gera uma árvore com o arquivo
        self.raiz = tree.getroot() #Define a raiz

        #Percorre os nós do arquivo, procurando todas as TAGS com esse nome
        #Nesse caso estamos percorrendo para pegar os valores da ViewPort
        for view in self.raiz.findall('viewport/vpmin'):
            vpmin = view.attrib #Atribui os atributos da TAG

        for view in self.raiz.findall('viewport/vpmax'):
            vpmax = view.attrib

        vpminX = float(vpmin.get('x')) #Pega o atributo X
        vpminY = float(vpmin.get('y')) #Pega o atributo Y
        vpmaxX = float(vpmax.get('x'))
        vpmaxY = float(vpmax.get('y'))


        #Vamos pegar agora os valores da Window
        for win in self.raiz.findall('window/wmin'):
            wmin = win.attrib

        for win in self.raiz.findall('window/wmax'):
            wmax = win.attrib

        wminX = float(wmin.get('x'))
        wminY = float(wmin.get('y'))
        wmaxX = float(wmax.get('x'))
        wmaxY = float(wmax.get('y'))

        self.janela = Window(wminX,wminY,wmaxX,wmaxY)                  #Criando a instância da Window
        self.viewport = Viewport(vpminX,vpminY,vpmaxX,vpmaxY)    #Criando a instância da Viewport

        #Pegando os valores dos pontos
        self.pontos = [] #Criando um vetor de Pontos
        for pon in self.raiz.findall('ponto'):
            ponto = pon.attrib
            ponX = float(ponto.get('x'))
            ponY = float(ponto.get('y'))
            self.pontos.append(Ponto(ponX,ponY))    #Adicionando os valores ao ponto

        #Pegando valores das Retas
        pontoRetas = [] #Criamos um vetor de pontos da reta
        self.retas = [] #Criamos um vetor com as retas

        for ret in self.raiz.findall('reta'):
            pontoRetas.clear()
            for ponto in ret:
                x = float(ponto.attrib.get('x'))
                y = float(ponto.attrib.get('y'))
                pontoRetas.append(Ponto(x,y)) #salvando os pontos da reta em um vetor
            reta = Reta(*pontoRetas)
            self.retas.append(reta) #salvando a reta em um vetor

        
        #Pegar os pontos do Poligono
        pontosPoligono = [] #Fazemos o mesmo esquema das retas
        self.poligonos = []
        for poli in self.raiz.findall('poligono'):
            pontosPoligono.clear()
            for ponto in poli:
                x = float(ponto.attrib.get('x'))
                y = float(ponto.attrib.get('y'))
                pontosPoligono.append(Ponto(x,y))
            poligono = Poligono(*pontosPoligono)
            self.poligonos.append(poligono)

In [9]:
class screen(QWidget):
    def __init__(self, parent=None):
        super().__init__(parent)
        # self.setMinimumSize(600,400)
        self.setFixedSize(620,460)

        self.pen = QPen()
        self.pen.setWidth(2)
        self.points = []
        self.lines = []
        self.polygons = []


    def paintEvent(self, event):
        painter = QPainter(self)

        painter.setBrush(QColor(0,255,255))
        painter.drawRect(self.rect())
        painter.setPen(self.pen)


        for point in self.points:
            painter.setBrush(QColor('black'))
            painter.drawEllipse(point, 2,2)
            painter.setBrush(Qt.NoBrush)
        for line in self.lines:
            painter.drawLine(line)
        for polygon in self.polygons:
            painter.setBrush(Qt.NoBrush)
            painter.drawPolygon(polygon)



class AddPointDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__()
        self.setWindowTitle("Add Point")
        self.setLayout(QFormLayout())
        self.x_input = QLineEdit()
        self.y_input = QLineEdit()
        self.layout().addRow("X:", self.x_input)
        self.layout().addRow("Y:", self.y_input)
        self.ok_button = QPushButton("OK")
        self.cancel_button = QPushButton("Cancel")
        self.ok_button.clicked.connect(self.accept)
        self.cancel_button.clicked.connect(self.reject)
        self.layout().addWidget(self.ok_button)
        self.layout().addWidget(self.cancel_button)

    def get_coordinates(self):
        try:
            x = float(self.x_input.text())
            y = float(self.y_input.text())
            return QPointF(x, y)
        except ValueError:
            return None
        
class AddLineDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__()
        self.setWindowTitle("Add Line")
        self.setLayout(QFormLayout())
        self.x1_input = QLineEdit()
        self.y1_input = QLineEdit()
        self.x2_input = QLineEdit()
        self.y2_input = QLineEdit()
        self.layout().addRow("X1:", self.x1_input)
        self.layout().addRow("Y1:", self.y1_input)
        self.layout().addRow("X2:", self.x2_input)
        self.layout().addRow("Y2:", self.y2_input)
        self.ok_button = QPushButton("OK")
        self.cancel_button = QPushButton("Cancel")
        self.ok_button.clicked.connect(self.accept)
        self.cancel_button.clicked.connect(self.reject)
        self.layout().addWidget(self.ok_button)
        self.layout().addWidget(self.cancel_button)

    def get_coordinates(self):
        try:
            x1 = float(self.x1_input.text())
            y1 = float(self.y1_input.text())
            x2 = float(self.x2_input.text())
            y2 = float(self.y2_input.text())
            return x1, y1, x2, y2
        except ValueError:
            return None

class AddPolygonDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__()
        self.setWindowTitle("Add Polygon")
        self.setLayout(QFormLayout())
        self.points_input = QLineEdit()
        self.layout().addRow("Points (x1,y1 x2,y2 ...):", self.points_input)
        self.ok_button = QPushButton("OK")
        self.cancel_button = QPushButton("Cancel")
        self.ok_button.clicked.connect(self.accept)
        self.cancel_button.clicked.connect(self.reject)
        self.layout().addWidget(self.ok_button)
        self.layout().addWidget(self.cancel_button)

    def get_coordinates(self):
        points_text = self.points_input.text()
        points = points_text.split()
        try:
            return [QPointF(float(p.split(',')[0]), float(p.split(',')[1])) for p in points]
        except ValueError:
            return None

In [10]:
class mainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        
        self.arq = "entrada.xml"
        self.file = LerXML(self.arq)
        self.trans = Transformada(self.file.janela, self.file.viewport)
        self.validate = False
        self.winPoint = []
        self.winLine = []
        self.winPolygons = []
        self.canva = screen()
        
        
        self.moveUp = QPushButton("Up", self)
        self.moveUp.clicked.connect(self.MoveUp)
        self.moveDown = QPushButton("Down", self)
        self.moveDown.clicked.connect(self.MoveDown)
        self.moveLeft = QPushButton("Left", self)
        self.moveLeft.clicked.connect(self.MoveLeft)
        self.moveRight = QPushButton("Right", self)
        self.moveRight.clicked.connect(self.MoveRight)
        self.moveRotate = QPushButton("Rotate-90", self)
        self.moveRotate.clicked.connect(self.Rotate)
        self.moveRotateInv = QPushButton("Rotate90", self)
        self.moveRotateInv.clicked.connect(self.RotateInv)
        self.diminuir = QPushButton("Diminuir", self)
        self.diminuir.clicked.connect(self.Diminuir)
        self.aumentar = QPushButton("Aumentar", self)
        self.aumentar.clicked.connect(self.Aumentar)
        
        centralWidget = QWidget()
        
        layout = QGridLayout(centralWidget)
        layout.addWidget(self.canva, 0, 0, 2, 3, alignment=Qt.AlignmentFlag.AlignCenter)
        layout.addWidget(self.moveUp, 2, 0, alignment=Qt.AlignmentFlag.AlignCenter)
        layout.addWidget(self.moveDown, 2, 1, alignment=Qt.AlignmentFlag.AlignCenter)
        layout.addWidget(self.moveLeft, 3, 0, alignment=Qt.AlignmentFlag.AlignCenter)
        layout.addWidget(self.moveRight, 3, 1, alignment=Qt.AlignmentFlag.AlignCenter)
        layout.addWidget(self.moveRotateInv, 3, 2, alignment=Qt.AlignmentFlag.AlignCenter)
        layout.addWidget(self.moveRotate, 2, 2, alignment=Qt.AlignmentFlag.AlignCenter)
        layout.addWidget(self.diminuir, 2, 3, alignment=Qt.AlignmentFlag.AlignCenter)
        layout.addWidget(self.aumentar, 3, 3, alignment=Qt.AlignmentFlag.AlignCenter)
        self.setCentralWidget(centralWidget)

        self.action_open = QAction("Open", self)
        self.file_menu = self.menuBar().addMenu("File")
        self.file_menu.addAction(self.action_open)
        self.action_open.triggered.connect(self.LerArq)


        self.file_menu = self.menuBar().addMenu("Add")
        self.action_add = QAction("Point", self)
        self.action_add.triggered.connect(self.show_add_point_dialog)
        self.file_menu.addAction(self.action_add)

        self.action_add = QAction("Line", self)
        self.file_menu.addAction(self.action_add)
        self.action_add.triggered.connect(self.show_add_line_dialog)
        
        self.action_add = QAction("Polygon", self)
        self.file_menu.addAction(self.action_add)
        self.action_add.triggered.connect(self.show_add_polygon_dialog)

        
        self.add_point_dialog = AddPointDialog(self)
        self.add_line_dialog = AddLineDialog(self)
        self.add_polygon_dialog = AddPolygonDialog(self)

    def show_add_point_dialog(self):
        if self.add_point_dialog.exec() == QDialog.Accepted:
            coords = self.add_point_dialog.get_coordinates()
            self.winPoint.append(Ponto(coords.x(), coords.y()))
            vcoords = self.trans.transformarPonto(coords.x(), coords.y())
            if vcoords:
                self.canva.points.append(QPointF(vcoords.x, vcoords.y))
                self.canva.update()
    
    def show_add_line_dialog(self):
        if self.add_line_dialog.exec() == QDialog.Accepted:
            coords = self.add_line_dialog.get_coordinates()
            if coords:
                x1, y1, x2, y2 = coords
                linePoints = Ponto(x1,y1), Ponto(x2, y2)
                self.winLine.append(linePoints)
                line = self.trans.transformarReta(linePoints)
                initLinePoint = line.pontosReta[0]
                endLinePoint = line.pontosReta[1]
                # line = QLineF(QPointF(x1, y1), QPointF(x2, y2))
                self.canva.lines.append(QLineF(QPointF(initLinePoint.x, initLinePoint.y), QPointF(endLinePoint.x, endLinePoint.y)))
                self.canva.update()
    
    def show_add_polygon_dialog(self):
        if self.add_polygon_dialog.exec() == QDialog.Accepted:
            points = self.add_polygon_dialog.get_coordinates()
            if points:
                polygons = []
                for point in points:
                    polygonPoint = Ponto(point.x(), point.y())
                    polygons.append(polygonPoint)
                    self.winPolygons.append(Poligono(polygonPoint))
                polygon = self.trans.transformarPoli(polygons)
                variable = []
                for pol in polygon:
                    variable.append(QPointF(pol.x, pol.y))                    
                self.canva.polygons.append(QPolygonF(variable))
                self.canva.update()

    def LerArq(self):
        self.arq = QFileDialog.getOpenFileName()[0]
        self.file = LerXML(self.arq)
        self.trans = Transformada(self.file.janela, self.file.viewport)
    
        for x in self.file.pontos:
            self.winPoint.append(Ponto(x.x, x.y))
            point = self.trans.transformarPonto(x.x, x.y)
            self.canva.points.append(QPointF(point.x, point.y))

        for retas in self.file.retas:
            linePoints = Ponto(retas.pontosReta[0].x, retas.pontosReta[0].y),Ponto(retas.pontosReta[1].x, retas.pontosReta[1].y)
            self.winLine.append(linePoints)
            line = self.trans.transformarReta(retas.pontosReta)
            initLinePoint = line.pontosReta[0]
            endLinePoint = line.pontosReta[1]
            self.canva.lines.append(QLineF(QPointF(initLinePoint.x,initLinePoint.y), QPointF(endLinePoint.x,endLinePoint.y)))

        for poli in self.file.poligonos:
            for polygonP in poli.pontosPoligono:
                self.winPolygons.append(Poligono(polygonP.x, polygonP.y))
            polygonPoints = self.trans.transformarPoli(poli.pontosPoligono)
            polygon = QPolygonF()
            variable = []
            for point in polygonPoints:
                variable.append(QPointF(point.x, point.y))
            self.canva.polygons.append(QPolygonF(variable))
            
        self.canva.update()

    def MoveUp(self):
        #self.file.janela.maxY += -1
        #self.file.janela.minY += -1

        #move = Transformada(self.file.janela, self.file.viewport)
        translation = np.array([
            [1, 0, 0],
            [0, 1, -0.5],
            [0, 0, 1]
        ])

        transformation_matrix = translation
        # Aplicar translação à janela (minX, minY) e (maxX, maxY)
        min_point = np.array([self.file.janela.minX, self.file.janela.minY, 1])
        max_point = np.array([self.file.janela.maxX, self.file.janela.maxY, 1])

        transladar_min_point = transformation_matrix @ min_point
        transladar_max_point = transformation_matrix @ max_point

        # Atualiza os limites da janela com os novos valores
        self.file.janela.minX, self.file.janela.minY = transladar_min_point[0], transladar_min_point[1]
        self.file.janela.maxX, self.file.janela.maxY = transladar_max_point[0], transladar_max_point[1]

        # Agora, cria uma nova transformação com a janela rotacionada
        move = Transformada(self.file.janela, self.file.viewport)
        
        updated_points = []
        for point in self.winPoint:
            point = move.transformarPonto(point.x, point.y)
            updated_points.append(QPointF(point.x, point.y))
        
        self.canva.points = updated_points

        updated_lines = []
        for line in self.winLine:
            line = move.transformarReta(line)
            initLinePoint = line.pontosReta[0]
            endLinePoint = line.pontosReta[1]
            updated_lines.append(QLineF(QPointF(initLinePoint.x,initLinePoint.y), QPointF(endLinePoint.x,endLinePoint.y)))

        self.canva.lines = updated_lines

        updated_polygons = []
        polygons = []
        for poly in self.winPolygons:
            poly = move.transformarPoli(poly.pontosPoligono)
            for polyTrans in poly:
                polygons.append(QPointF(polyTrans.x, polyTrans.y))
        updated_polygons.append(QPolygonF(polygons))

        self.canva.polygons = updated_polygons
        self.canva.update()

    def MoveDown(self):
        #self.file.janela.maxY += 1
        #self.file.janela.minY += 1

        #move = Transformada(self.file.janela, self.file.viewport)
        translation = np.array([
            [1, 0, 0],
            [0, 1, 0.5],
            [0, 0, 1]
        ])

        transformation_matrix = translation
        # Aplicar translação à janela (minX, minY) e (maxX, maxY)
        min_point = np.array([self.file.janela.minX, self.file.janela.minY, 1])
        max_point = np.array([self.file.janela.maxX, self.file.janela.maxY, 1])

        transladar_min_point = transformation_matrix @ min_point
        transladar_max_point = transformation_matrix @ max_point

        # Atualiza os limites da janela com os novos valores
        self.file.janela.minX, self.file.janela.minY = transladar_min_point[0], transladar_min_point[1]
        self.file.janela.maxX, self.file.janela.maxY = transladar_max_point[0], transladar_max_point[1]

        # Agora, cria uma nova transformação com a janela rotacionada
        move = Transformada(self.file.janela, self.file.viewport)

        updated_points = []
        for point in self.winPoint:
            point = move.transformarPonto(point.x, point.y)
            updated_points.append(QPointF(point.x, point.y))
        
        self.canva.points = updated_points

        updated_lines = []
        for line in self.winLine:
            line = move.transformarReta(line)
            initLinePoint = line.pontosReta[0]
            endLinePoint = line.pontosReta[1]
            updated_lines.append(QLineF(QPointF(initLinePoint.x,initLinePoint.y), QPointF(endLinePoint.x,endLinePoint.y)))

        self.canva.lines = updated_lines
        self.canva.update()
        
        updated_polygons = []
        polygons = []
        for poly in self.winPolygons:
            poly = move.transformarPoli(poly.pontosPoligono)
            for polyTrans in poly:
                polygons.append(QPointF(polyTrans.x, polyTrans.y))
        updated_polygons.append(QPolygonF(polygons))

        self.canva.polygons = updated_polygons
        self.canva.update()
    
    def MoveLeft(self):
        #self.file.janela.maxX += 1
        #self.file.janela.minX += 1

        #move = Transformada(self.file.janela, self.file.viewport)
        translation = np.array([
            [1, 0, 0.5],
            [0, 1, 0],
            [0, 0, 1]
        ])

        transformation_matrix = translation
        # Aplicar translação à janela (minX, minY) e (maxX, maxY)
        min_point = np.array([self.file.janela.minX, self.file.janela.minY, 1])
        max_point = np.array([self.file.janela.maxX, self.file.janela.maxY, 1])

        transladar_min_point = transformation_matrix @ min_point
        transladar_max_point = transformation_matrix @ max_point

        # Atualiza os limites da janela com os novos valores
        self.file.janela.minX, self.file.janela.minY = transladar_min_point[0], transladar_min_point[1]
        self.file.janela.maxX, self.file.janela.maxY = transladar_max_point[0], transladar_max_point[1]

        # Agora, cria uma nova transformação com a janela rotacionada
        move = Transformada(self.file.janela, self.file.viewport)

        updated_points = []
        for point in self.winPoint:
            point = move.transformarPonto(point.x, point.y)
            updated_points.append(QPointF(point.x, point.y))
        
        self.canva.points = updated_points

        updated_lines = []
        for line in self.winLine:
            line = move.transformarReta(line)
            initLinePoint = line.pontosReta[0]
            endLinePoint = line.pontosReta[1]
            updated_lines.append(QLineF(QPointF(initLinePoint.x,initLinePoint.y), QPointF(endLinePoint.x,endLinePoint.y)))

        self.canva.lines = updated_lines

        updated_polygons = []
        polygons = []
        for poly in self.winPolygons:
            poly = move.transformarPoli(poly.pontosPoligono)
            for polyTrans in poly:
                polygons.append(QPointF(polyTrans.x, polyTrans.y))
        updated_polygons.append(QPolygonF(polygons))

        self.canva.polygons = updated_polygons
        self.canva.update()
        
    def MoveRight(self):
        #self.file.janela.maxX += -1
        #self.file.janela.minX += -1

        #move = Transformada(self.file.janela, self.file.viewport)
        translation = np.array([
            [1, 0, -0.5],
            [0, 1, 0],
            [0, 0, 1]
        ])

        transformation_matrix = translation
        # Aplicar translação à janela (minX, minY) e (maxX, maxY)
        min_point = np.array([self.file.janela.minX, self.file.janela.minY, 1])
        max_point = np.array([self.file.janela.maxX, self.file.janela.maxY, 1])

        transladar_min_point = transformation_matrix @ min_point
        transladar_max_point = transformation_matrix @ max_point

        # Atualiza os limites da janela com os novos valores
        self.file.janela.minX, self.file.janela.minY = transladar_min_point[0], transladar_min_point[1]
        self.file.janela.maxX, self.file.janela.maxY = transladar_max_point[0], transladar_max_point[1]

        # Agora, cria uma nova transformação com a janela rotacionada
        move = Transformada(self.file.janela, self.file.viewport)
        
        updated_points = []
        for point in self.winPoint:
            point = move.transformarPonto(point.x, point.y)
            updated_points.append(QPointF(point.x, point.y))
        
        self.canva.points = updated_points

        updated_lines = []
        for line in self.winLine:
            line = move.transformarReta(line)
            initLinePoint = line.pontosReta[0]
            endLinePoint = line.pontosReta[1]
            updated_lines.append(QLineF(QPointF(initLinePoint.x,initLinePoint.y), QPointF(endLinePoint.x,endLinePoint.y)))

        self.canva.lines = updated_lines

        updated_polygons = []
        polygons = []
        for poly in self.winPolygons:
            poly = move.transformarPoli(poly.pontosPoligono)
            for polyTrans in poly:
                polygons.append(QPointF(polyTrans.x, polyTrans.y))
        updated_polygons.append(QPolygonF(polygons))

        self.canva.polygons = updated_polygons
        self.canva.update()
        
    def Rotate(self):
        # Ângulo de rotação em radianos
        angle = np.radians(90)

        # Matriz de rotação de 45 graus
        scale_matriz = np.array([
            [np.cos(angle), -np.sin(angle), 0],
            [np.sin(angle), np.cos(angle), 0],
            [0, 0, 1]
        ])

        # Calcula o centro da janela
        center_x = (self.file.janela.minX + self.file.janela.maxX) / 2
        center_y = (self.file.janela.minY + self.file.janela.maxY) / 2

        # Matriz de translação para mover o centro para a origem
        translation_to_origin = np.array([
            [1, 0, -center_x],
            [0, 1, -center_y],
            [0, 0, 1]
        ])

        # Matriz de translação para mover o centro de volta à posição original
        translation_back = np.array([
            [1, 0, center_x],
            [0, 1, center_y],
            [0, 0, 1]
        ])

        # Matriz de transformação composta: Translação para origem -> Rotação -> Translação de volta
        transformation_matrix = translation_back @ scale_matriz @ translation_to_origin

        # Aplicar rotação à janela (minX, minY) e (maxX, maxY)
        min_point = np.array([self.file.janela.minX, self.file.janela.minY, 1])
        max_point = np.array([self.file.janela.maxX, self.file.janela.maxY, 1])

        rotated_min_point = transformation_matrix @ min_point
        rotated_max_point = transformation_matrix @ max_point

        # Atualiza os limites da janela com os novos valores
        self.file.janela.minX,self.file.janela.minY = rotated_min_point[0], rotated_min_point[1]
        self.file.janela.maxX, self.file.janela.maxY = rotated_max_point[0], rotated_max_point[1]
        
        

        # Agora, cria uma nova transformação com a janela rotacionada
        move = Transformada(self.file.janela, self.file.viewport)
            
        updated_points = []
        for point in self.winPoint:
            point = move.transformarPonto(point.x, point.y)
            updated_points.append(QPointF(point.x, point.y))

        self.canva.points = updated_points

        updated_lines = []
        for line in self.winLine:
            line = move.transformarReta(line)
            initLinePoint = line.pontosReta[0]
            endLinePoint = line.pontosReta[1]
            updated_lines.append(QLineF(QPointF(initLinePoint.x, initLinePoint.y), QPointF(endLinePoint.x, endLinePoint.y)))

        self.canva.lines = updated_lines

        updated_polygons = []
        polygons = []
        for poly in self.winPolygons:
            poly = move.transformarPoli(poly.pontosPoligono)
            for polyTrans in poly:
                polygons.append(QPointF(polyTrans.x, polyTrans.y))
        updated_polygons.append(QPolygonF(polygons))

        self.canva.polygons = updated_polygons
        self.canva.update()
        
    def RotateInv(self):
        # Ângulo de rotação em radianos
        angle = np.radians(-90)

        # Matriz de rotação de 45 graus
        scale_matriz = np.array([
            [np.cos(angle), -np.sin(angle), 0],
            [np.sin(angle), np.cos(angle), 0],
            [0, 0, 1]
        ])

        # Calcula o centro da janela
        center_x = (self.file.janela.minX + self.file.janela.maxX) / 2
        center_y = (self.file.janela.minY + self.file.janela.maxY) / 2

        # Matriz de translação para mover o centro para a origem
        translation_to_origin = np.array([
            [1, 0, -center_x],
            [0, 1, -center_y],
            [0, 0, 1]
        ])

        # Matriz de translação para mover o centro de volta à posição original
        translation_back = np.array([
            [1, 0, center_x],
            [0, 1, center_y],
            [0, 0, 1]
        ])

        # Matriz de transformação composta: Translação para origem -> Rotação -> Translação de volta
        transformation_matrix = translation_back @ scale_matriz @ translation_to_origin

        # Aplicar rotação à janela (minX, minY) e (maxX, maxY)
        min_point = np.array([self.file.janela.minX, self.file.janela.minY, 1])
        max_point = np.array([self.file.janela.maxX, self.file.janela.maxY, 1])

        rotated_min_point = transformation_matrix @ min_point
        rotated_max_point = transformation_matrix @ max_point

        # Atualiza os limites da janela com os novos valores
        self.file.janela.minX,self.file.janela.minY = rotated_min_point[0], rotated_min_point[1]
        self.file.janela.maxX, self.file.janela.maxY = rotated_max_point[0], rotated_max_point[1]
        
        

        # Agora, cria uma nova transformação com a janela rotacionada
        move = Transformada(self.file.janela, self.file.viewport)
            
        updated_points = []
        for point in self.winPoint:
            point = move.transformarPonto(point.x, point.y)
            updated_points.append(QPointF(point.x, point.y))

        self.canva.points = updated_points

        updated_lines = []
        for line in self.winLine:
            line = move.transformarReta(line)
            initLinePoint = line.pontosReta[0]
            endLinePoint = line.pontosReta[1]
            updated_lines.append(QLineF(QPointF(initLinePoint.x, initLinePoint.y), QPointF(endLinePoint.x, endLinePoint.y)))

        self.canva.lines = updated_lines

        updated_polygons = []
        polygons = []
        for poly in self.winPolygons:
            poly = move.transformarPoli(poly.pontosPoligono)
            for polyTrans in poly:
                polygons.append(QPointF(polyTrans.x, polyTrans.y))
        updated_polygons.append(QPolygonF(polygons))

        self.canva.polygons = updated_polygons
        self.canva.update()
        
    def Diminuir(self):

        scale_matriz = np.array([
            [1.5, 0, 0],
            [0, 1.5, 0],
            [0, 0, 1]
        ])

        # Calcula o centro da janela
        center_x = (self.file.janela.minX + self.file.janela.maxX) / 2
        center_y = (self.file.janela.minY + self.file.janela.maxY) / 2

        # Matriz de translação para mover o centro para a origem
        translation_to_origin = np.array([
            [1, 0, -center_x],
            [0, 1, -center_y],
            [0, 0, 1]
        ])

        # Matriz de translação para mover o centro de volta à posição original
        translation_back = np.array([
            [1, 0, center_x],
            [0, 1, center_y],
            [0, 0, 1]
        ])

        # Matriz de transformação composta: Translação para origem -> Rotação -> Translação de volta
        transformation_matrix = translation_back @ scale_matriz @ translation_to_origin

        # Aplicar rotação à janela (minX, minY) e (maxX, maxY)
        min_point = np.array([self.file.janela.minX, self.file.janela.minY, 1])
        max_point = np.array([self.file.janela.maxX, self.file.janela.maxY, 1])

        scale_min_point = transformation_matrix @ min_point
        scale_max_point = transformation_matrix @ max_point

        # Atualiza os limites da janela com os novos valores
        self.file.janela.minX,self.file.janela.minY = scale_min_point[0], scale_min_point[1]
        self.file.janela.maxX, self.file.janela.maxY = scale_max_point[0], scale_max_point[1]

        # Agora, cria uma nova transformação com a janela rotacionada
        move = Transformada(self.file.janela, self.file.viewport)
            
        updated_points = []
        for point in self.winPoint:
            point = move.transformarPonto(point.x, point.y)
            updated_points.append(QPointF(point.x, point.y))

        self.canva.points = updated_points

        updated_lines = []
        for line in self.winLine:
            line = move.transformarReta(line)
            initLinePoint = line.pontosReta[0]
            endLinePoint = line.pontosReta[1]
            updated_lines.append(QLineF(QPointF(initLinePoint.x, initLinePoint.y), QPointF(endLinePoint.x, endLinePoint.y)))

        self.canva.lines = updated_lines

        updated_polygons = []
        polygons = []
        for poly in self.winPolygons:
            poly = move.transformarPoli(poly.pontosPoligono)
            for polyTrans in poly:
                polygons.append(QPointF(polyTrans.x, polyTrans.y))
        updated_polygons.append(QPolygonF(polygons))

        self.canva.polygons = updated_polygons
        self.canva.update()
        
    def Aumentar(self):

        scale_matriz = np.array([
            [1/1.5, 0, 0],
            [0, 1/1.5, 0],
            [0, 0, 1]
        ])

        # Calcula o centro da janela
        center_x = (self.file.janela.minX + self.file.janela.maxX) / 2
        center_y = (self.file.janela.minY + self.file.janela.maxY) / 2

        # Matriz de translação para mover o centro para a origem
        translation_to_origin = np.array([
            [1, 0, -center_x],
            [0, 1, -center_y],
            [0, 0, 1]
        ])

        # Matriz de translação para mover o centro de volta à posição original
        translation_back = np.array([
            [1, 0, center_x],
            [0, 1, center_y],
            [0, 0, 1]
        ])

        # Matriz de transformação composta: Translação para origem -> Rotação -> Translação de volta
        transformation_matrix = translation_back @ scale_matriz @ translation_to_origin

        # Aplicar rotação à janela (minX, minY) e (maxX, maxY)
        min_point = np.array([self.file.janela.minX, self.file.janela.minY, 1])
        max_point = np.array([self.file.janela.maxX, self.file.janela.maxY, 1])

        scale_min_point = transformation_matrix @ min_point
        scale_max_point = transformation_matrix @ max_point

        # Atualiza os limites da janela com os novos valores
        self.file.janela.minX,self.file.janela.minY = scale_min_point[0], scale_min_point[1]
        self.file.janela.maxX, self.file.janela.maxY = scale_max_point[0], scale_max_point[1]

        # Agora, cria uma nova transformação com a janela rotacionada
        move = Transformada(self.file.janela, self.file.viewport)
            
        updated_points = []
        for point in self.winPoint:
            point = move.transformarPonto(point.x, point.y)
            updated_points.append(QPointF(point.x, point.y))

        self.canva.points = updated_points

        updated_lines = []
        for line in self.winLine:
            line = move.transformarReta(line)
            initLinePoint = line.pontosReta[0]
            endLinePoint = line.pontosReta[1]
            updated_lines.append(QLineF(QPointF(initLinePoint.x, initLinePoint.y), QPointF(endLinePoint.x, endLinePoint.y)))

        self.canva.lines = updated_lines

        updated_polygons = []
        polygons = []
        for poly in self.winPolygons:
            poly = move.transformarPoli(poly.pontosPoligono)
            for polyTrans in poly:
                polygons.append(QPointF(polyTrans.x, polyTrans.y))
        updated_polygons.append(QPolygonF(polygons))

        self.canva.polygons = updated_polygons
        self.canva.update()
        

In [11]:
from PySide6.QtWidgets import QApplication
import sys

app = QApplication([])
window = mainWindow()

window.show()

sys.exit(app.exec())

SystemExit: 0

C:\Users\gusta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
